In [1]:
import os

In [2]:
os.getcwd()

'/Users/psykick/Documents/learning/github/text-summarization-end-to-end/research'

In [3]:
os.chdir('..')

In [5]:
from dataclasses import dataclass
from pathlib import Path

# this class will serve as a datatype for the data ingestion configuration
@dataclass(frozen=True)
class DataIngestionConfig:
    """
    DataIngestionConfig class is used to store the configuration for the data ingestion process.
    """
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from text_summarization.constants import *
from text_summarization.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    """
    Configuration Manager class to read the configuration and parameters files for each module
    """
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        """Constructor
        
        Args:
            config_filepath (_type_, optional): path to the configuration file. Defaults to CONFIG_FILE_PATH.
            params_filepath (_type_, optional): path to the parameters file. Defaults to PARAMS_FILE_PATH.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)->DataIngestionConfig:
        """Get the data ingestion configuration

        Returns:
            DataIngestionConfig: data ingestion configuration
        """
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        return DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_url = config.source_url,
            local_data_file = Path(config.local_data_file),
            unzip_dir = Path(config.unzip_dir)
        )

In [14]:
import os
import urllib.request as request
import zipfile
from text_summarization.logging import logger
from text_summarization.utils.common import get_size

In [15]:
class DataIngestion:
    """
    This class is responsible for downloading and extracting the data from the source URL
    """
    def __init__(self, config: DataIngestionConfig):
        """Initializes the DataIngestion class

        Args:
            config (DataIngestionConfig): DataIngestionConfig object
        """
        self.config = config
    
    def download_file(self):
        """
        Downloads the file from the source URL if it doesn't exist
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloaded file: {filename} with info:\n{headers}")
        else:
            logger.info(f"File already exists: {self.config.local_data_file}\nSize: {get_size(self.config.local_data_file)}")
    
    def extract_zip_file(self):
        """
        Extracts the zip file to the specified directory
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
# data ingestion pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"Data Ingestion failed: {str(e)}")
    raise e

[2024-04-14 15:20:18,635] [INFO] [common] [common.py:24] yaml file: config/config.yaml loaded successfully
[2024-04-14 15:20:18,636] [INFO] [common] [common.py:24] yaml file: params.yaml loaded successfully
[2024-04-14 15:20:18,637] [INFO] [common] [common.py:40] created directory at: artifacts
[2024-04-14 15:20:18,637] [INFO] [common] [common.py:40] created directory at: artifacts/data_ingestion
[2024-04-14 15:20:20,599] [INFO] [161847061] [161847061.py:11] Downloaded file: artifacts/data_ingestion/data.zip with info:
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3084:354996:1BF0B5:25A026:661BA6DB
Accept-Ranges: bytes
Date: Sun, 14 Ap